In [1]:
import requests
import pandas as pd

def scrape():
    url = "https://gitlab.com/api/v4/projects/7764/"

    commits = requests.request("GET", url+"repository/commits", headers={}, data={}).json()
    
    """
    Future data pipelines to be created, implemented "commits" for now

    pipelines = requests.request("GET", url+"pipelines", headers={}, data={}).json()
    jobs = requests.request("GET", url+"jobs", headers={}, data={}).json()
    branches = requests.request("GET", url+"repository/branches", headers={}, data={}).json()
    merge_requests = requests.request("GET", url+"merge_requests", headers={}, data={}).json()
    pipeline_schedules = requests.request("GET", url+"pipeline_schedules", headers={}, data={}).json()
    pipeline_triggers = requests.request("GET", url+"triggers", headers={}, data={}).json()
    issues = requests.request("GET", url + "issues", headers={}, data={})
    issues_open = requests.request("GET", url + "issues?state=open", headers={}, data={})
    issues_closed = requests.request("GET", url + "issues?state=closed", headers={}, data={})
    """
    
    return commits

In [2]:
commits_jsonlist = scrape()

In [3]:
commits_df = pd.DataFrame(commits_jsonlist)

In [4]:
commits_df = commits_df.drop(commits_df.columns.difference(['committed_date']), axis=1)

In [5]:
# Temporary measure before time zone conversion is implemented
for (index, row) in commits_df.iterrows():
    if (row[0][23:]!="+00:00"):
        commits_df = commits_df.drop(commits_df.index[index])

In [6]:
commits_df

,committed_date
0,2022-06-30T13:16:32.000+00:00
1,2022-06-30T12:18:25.000+00:00
2,2022-06-30T12:18:21.000+00:00
3,2022-06-30T12:08:44.000+00:00
4,2022-06-30T12:08:43.000+00:00
5,2022-06-30T08:13:26.000+00:00
6,2022-06-30T08:13:21.000+00:00
7,2022-06-30T08:13:02.000+00:00
8,2022-06-30T08:12:59.000+00:00
9,2022-06-30T07:55:09.000+00:00


In [7]:
aggregated_commits_df = pd.DataFrame({'Time': [], 'Number of commits': []})

In [8]:
current_row = True
for (index, row) in commits_df.iterrows():
    if (index==0):
        hour = row[0][0:13]
        n = 1
    elif (current_row==False):
        hour = row[0][0:13]
        n = 2
        current_row = True
    else:
        if (row[0][0:13]==hour):
            n += 1
        else:
            current_row = False
            aggregated_commits_df = aggregated_commits_df.append({'Time': hour, 'Number of commits': n}, ignore_index=True)
aggregated_commits_df = aggregated_commits_df.append({'Time': row[0][0:13], 'Number of commits': (n-1)}, ignore_index=True)

In [9]:
aggregated_commits_df

,Time,Number of commits
0,2022-06-30T13,1.0
1,2022-06-30T12,4.0
2,2022-06-30T08,4.0
3,2022-06-30T07,2.0
4,2022-06-30T06,2.0
5,2022-06-30T04,6.0
6,2022-06-30T01,5.0


In [13]:
aggregated_commits_df.to_json('aggregated commits.json')